# OLS

1. [closed form solution](#closed-form-solution)
1. [interpreting results](#interpreting-results)
    - [degree of freedoms](#degree-of-freedoms)
    - [covariance type](#covariance-type)
    - [variances](#variances)
        - [$r^2$](#r-squared)
        - [tss, ess, rss, and their degree of freedom](#tss-ess-rss-and-their-dof)
        - [adjusted $r^2$](#adjusted-r-squared)
    - [likelihood](#likelihood)
        - [log likelihood](#log-likelihood)
        - [AIC](#aic)
        - [BIC](#bic)
    - [regression coefficient](#regression-coefficient)
        - [SE($\hat{\beta}$)](#standard-error-of-regression-coefficient)
        - [t-statistic for H0: $\beta=0$](#t-statistic-of-regression-coefficient)
        - [p-value for H0: $\beta=0$](#p-value-of-regression-coefficient-significance)
        - [CI for $\beta$](#confidence-interval-of-regression-coefficient)
        - [F-statistic, and probability of F-statistic](#f-statistic-and-probability-of-f-statistic)
    - [normality tests: omnibus, jaque bera test, skew, kurtosis](#normality-tests)
    - [multicollinearity](#multicollinearity)
        - [condition number](#condition-number)

1. [resources](#resources)

## Closed form solution

OLS finds the weight that minimize the residual sum of squares


\begin{align}
\mathbf{ \min_{\beta} L=(y-X\beta)^T(y-X\beta)}
\end{align}




We can get the closed form solution by setting the first derivative of the above loss function with respect to $\beta$ to 0.


\begin{align}
\mathbf{ \frac{dL}{d\beta}} & = \mathbf{2X^T(y-X\beta)} = 0\\
& \mathbf{ \hat{\beta}=(X^TX)^{-1}X^Ty}
\end{align}


Next, we'll apply this solution to a toy dataset.

In [1]:
import numpy as np
from sklearn.datasets import load_diabetes

In [2]:
data = load_diabetes()
data.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])

In [3]:
X = data.data
y = data.target
print(data.feature_names)
print(X.shape)

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
(442, 10)


We need to add a column of 1s to `X` for the intercept.

In [4]:
X = np.hstack([np.ones_like(y)[:, np.newaxis], X])

In [5]:
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
beta_hat

array([ 152.13348416,  -10.0098663 , -239.81564367,  519.84592005,
        324.3846455 , -792.17563855,  476.73902101,  101.04326794,
        177.06323767,  751.27369956,   67.62669218])

To validate the results above, we'll run the data through `statsmodels.api.OLS`

In [6]:
import statsmodels.api as sm

model = sm.OLS(y, X)
result = model.fit()
result.params

array([ 152.13348416,  -10.0098663 , -239.81564367,  519.84592005,
        324.3846455 , -792.17563855,  476.73902101,  101.04326794,
        177.06323767,  751.27369956,   67.62669218])

Looking good!

In real life, we don't need to manually calculate the closed-form solution, but it is extremely important to understand each component of the OLS output.

## Interpreting Results

In [7]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.27
Date:                Fri, 26 Apr 2024   Prob (F-statistic):           3.83e-62
Time:                        14:16:11   Log-Likelihood:                -2386.0
No. Observations:                 442   AIC:                             4794.
Df Residuals:                     431   BIC:                             4839.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.576     59.061      0.000     147.071     157.196
x1           -10.0099     59.749     -0.168      0.867    -127.446     107.426
x2          -239.8156     61.222     -3.917      0.000    -360.147    -119.484
x3           519.8459     66.533      7.813      0.000     389.076     650.616
x4           324.3846     65.422      4.958      0.000     195.799     452.970
x5          -792.1756    416.680     -1.901      0.058   -1611.153      26.802
x6           476.7390    339.030      1.406      0.160    -189.620    1143.098
x7           101.0433    212.531      0.475      0.635    -316.684     518.770
x8           177.0632    161.476      1.097      0.273    -140.315     494.441
x9           751.2737    171.900      4.370      0.000     413.407    1089.140
x10           67.6267     65.984      1.025      0.306     -62.064     197.318
==============================================================================
Omnibus:                        1.506   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.471   Jarque-Bera (JB):                1.404
Skew:                           0.017   Prob(JB):                        0.496
Kurtosis:                       2.726   Cond. No.                         227.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Degree of Freedoms

In [8]:
n = len(y)
p = len(beta_hat)

dof_model = p - 1 # drop 1 for beta_0
dof_rss = n - p # lost p dof estimating betas

print(f"number of observations: {n}")
print(f"df residuls: {dof_rss}")
print(f"df model: {dof_model}")

number of observations: 442
df residuls: 431
df model: 10


### Covariance Type

There're 2 types of covariance estimation
- non-robust, assuming homoskedasticity (equal variance) in residual 
- robust, hetereoskedasticity-robust

OLS assumes homoskedasticity, so non-robust covariance type is used by default.

Read more about this topic
- [Heteroskedasticity-consistent standard errors](https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors)
- [Set covariance type in `statsmodels`](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.get_robustcov_results.html)
- [Change covariance type to robust in statsmodels.logit](https://stackoverflow.com/questions/61171429/change-covariance-type-to-robust-in-statsmodels-logit)

### Variances

#### R-squared

$R^2$, or the coefficient of determination is calculated as

\begin{align}
\mathbf{R^2} & = \mathbf{\frac{ESS}{TSS}} = \mathbf{1 - \frac{RSS}{TSS}} 
\end{align}

#### TSS, ESS, RSS, and their DoF

Total sum of squares (TSS), explained sum of squares (ESS), and residual sum of squares (RSS) are defiend as


\begin{align}
\mathbf{TTS} & = \sum_i(y_i-\bar{y})^2, & \mathbf{dof_{TTS}} = n - 1\\
\mathbf{ESS} & = \sum_i(\hat{y}_i-\bar{y})^2, & \mathbf{dof_{ESS}} = n - p - 1\\
\mathbf{RSS} & = \sum_i(y_i-\hat{y}_i)^2, & \mathbf{dof_{RSS}} = n - p\\
\mathbf{TTS} & = \mathbf{ESS} + \mathbf{RSS}
\end{align}

From the equations above, we can see that,
- TSS is proportional to the variance in y. 
- ESS is proportional to the vairiance in $\hat{y}$.

$R^2$ shows the proportions of variance in y explained by the model fitted on X.



In [9]:
y_hat = X @ beta_hat
y_bar = np.mean(y)

rss = np.sum((y - y_hat) ** 2)
ess = np.sum((y_hat - y_bar) ** 2)
tss = np.sum((y - y_bar) ** 2)

print(f"rss {rss:.2f}")
print(f"ess {ess:.2f}")
print(f"rss + ess {rss + ess:.2f}")
print(f"tss {tss: .2f}")
print(f"r^2: {ess / tss * 100:.2f}%")

rss 1263985.79
ess 1357023.34
rss + ess 2621009.12
tss  2621009.12
r^2: 51.77%


#### Adjusted R squared

It's tempting to introduce as many features as possible to explain away the variance in y. Adjusted $r^2$ is introduced to penalize the number of features used in the fitted model.

\begin{align}
\mathbf{R_{adj}^2} & = 1 - \mathbf{\frac{RSS /dof_{rss}}{TSS / dof_{tss}}}
\end{align}

In [10]:
n = len(y)
p = len(beta_hat)

dof_tss = n - 1 # lost 1 dof estimating y bar
dof_rss = n - p # lost p dof estimating betas
r2_adj = 1 - (rss / tss) / (dof_rss / dof_tss)
print(f"r2_adj: {r2_adj * 100:.2f}%")
print(f"dof_tss: {dof_tss}")
print(f"dof_rss: {dof_rss}")

r2_adj: 50.66%
dof_tss: 441
dof_rss: 431


### Likelihood

The modeling hypothesis of OLS, h, is 
\begin{align}
\mathbf{e = y - \hat{y}} \sim \mathcal{N}(0, \sigma^2)
\end{align}

#### Log Likelihood

The log likelihood of observing y given the feature set and modeling hypothesis
\begin{align}
\mathbf{\log p(y | X; h)} = \sum_i \log \mathcal{N}(e_i | \mu, \sigma)
\end{align}

We can calculate the sample mean and variance of residual, and use the gaussian pdf $f(x)=\frac{1}{\sigma \sqrt{2\pi}}e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ to calculate the log likelihood.




In [11]:
e = y - y_hat
mu = np.mean(e)
sigma = np.std(e)
print(f"mu: {mu:.2f}")
print(f"sigma: {sigma:.2f}")

likelihood = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-1/2 * ((e - mu) / sigma) ** 2)
ll = np.sum(np.log(likelihood))
print(f"log likelihood: {ll:.1f}")

mu: 0.00
sigma: 53.48
log likelihood: -2386.0


Alternatively, we can use `scipy.stats.norm` to help us estimate the parameters, and calculate the log likelihood 

In [12]:
from scipy.stats import norm

e = y - y_hat
mu, sigma = norm.fit(e)
print(f"mu: {mu:.2f}")
print(f"sigma: {sigma:.2f}")

likelihood = norm.pdf(e, mu, sigma)
ll = np.sum(np.log(likelihood))
print(f"log likelihood: {ll:.1f}")

mu: 0.00
sigma: 53.48
log likelihood: -2386.0


#### AIC

Akaike information criterion, is defined as $- 2*ll + 2p$

#### BIC

Bayesian information criterion, is defined as $- 2*ll + \ln(n)p$

In [13]:
n = len(y)
p = len(beta_hat)

aic = - 2 * ll + 2 * p
bic = - 2 * ll + np.log(n) * p
print(f"aic: {aic:.0f}")
print(f"bic: {bic:.0f}")

aic: 4794
bic: 4839


Log likelhood (or negative log likelihood), AIC, and BIC are used in comparing models. Ideally we want to,

- maximize log likelihood
- minimize negative log likelihood
- minimize AIC
- minimize BIC

### Regression Coefficient

#### Standard Error of Regression Coefficient


One of OLS assumption is normality of residuals, which can be extend to the distribution of $\beta$ .

\begin{align}
e & \sim \mathcal{N} (0, \sigma^2I)\\
\mathbf{X\beta - X\hat{\beta}} & \sim \mathcal{N} (0, \sigma^2I) \\
\mathbf{\beta - \hat{\beta}} & \sim \mathcal{N} (0, \sigma^2(X^TX)^{-1}) \\
\mathbf{ \beta} & \sim \mathcal{N} (\hat{\beta}, \sigma^2(X^TX)^{-1})
\end{align}


Note that $\mathbf{Var(AX)=A^TVar(X)A}$, see [variance propagation rule](https://en.wikipedia.org/wiki/Variance#Propagation)

With the normality assumption of $\beta$, we can 
- compute the confidence interval, 
- and use t distribution (because $\sigma$ is unknown) to test H0: $\beta=0$

In [14]:
from numpy.testing import assert_almost_equal

e = y - y_hat
var_e = np.var(e, ddof=p) # lost degree of freedom estimating betas
cov_beta = var_e * np.linalg.inv(X.T @ X)
var_beta = np.diagonal(cov_beta)
se_beta = np.sqrt(var_beta)

print(se_beta)
assert_almost_equal(se_beta, result.bse, decimal=10)


[  2.57585449  59.74924652  61.22234394  66.53344474  65.42199205
 416.67987034 339.03049482 212.53145672 161.4757952  171.89998192
  65.98428191]


#### T-statistic of Regression Coefficient

In [15]:
t_val = (beta_hat - 0) / se_beta

print(t_val)
# comparing it to statsmodels OLS result
assert_almost_equal(t_val, result.tvalues, decimal=10)

[59.06136587 -0.16753126 -3.91712614  7.81330235  4.95834253 -1.90116129
  1.4061833   0.47542735  1.09653114  4.37041174  1.02489093]


#### P-value of Regression Coefficient Significance

In [16]:
from scipy.stats import t

# t.sf = 1 - t.cdf
# p-value is the probability of observing something more extreme on both tails given H0: \beta = 0 is true 
p_val = t.sf(np.abs(t_val), n-p) * 2

print(p_val)
# comparing it to statsmodels OLS result
assert_almost_equal(p_val, result.pvalues, decimal=10)


[1.01008193e-208 8.67030634e-001 1.04167119e-004 4.29639142e-014
 1.02427839e-006 5.79476054e-002 1.60390240e-001 6.34723256e-001
 2.73458694e-001 1.55589909e-005 3.05989526e-001]


#### Confidence Interval of Regression Coefficient

In [17]:
# t.ppf = inverse of t.cdf
beta_lb = beta_hat + t.ppf(0.025, df=n-p) * se_beta
beta_ub = beta_hat - t.ppf(0.025, df=n-p) * se_beta
beta_conf_int = np.hstack([beta_lb[:, np.newaxis], beta_ub[:, np.newaxis]])

print(beta_conf_int)
# comparing it to statsmodels OLS result
assert_almost_equal(beta_conf_int, result.conf_int(), decimal=10)

[[  147.07068514   157.19628319]
 [ -127.44601374   107.42628114]
 [ -360.14713953  -119.48414782]
 [  389.07554418   650.61629593]
 [  195.79881133   452.97047967]
 [-1611.15297363    26.80169652]
 [ -189.61976165  1143.09780366]
 [ -316.6837653    518.77030117]
 [ -140.31474443   494.44121978]
 [  413.40715232  1089.14024679]
 [  -62.06431331   197.31769768]]


#### F-statistic, and Probability of F-statistic

F-test is used when comparing variances. In the context of OLS, we're comparing a model forecasting $\bar{y}$ for every observation, against a model forecasting $\mathbf{X\beta}$.

Another way to interpret this is H0: $\mathbf{\beta^T}=[0, ..., 0]$


$\mathbf{F = \frac{\text{unexplained variance}}{\text{explained variance}}=\frac{rss_1-rss_2}{dof_1 - dof_2} / \frac{rss_2}{dof_2}}$

- $\mathbf{rss_1}$ and $\mathbf{dof_1}$ are the residual sum of square and degree of freedom of the reduced model

- $\mathbf{rss_2}$ and $\mathbf{dof_2}$ are the residual sum of square and degree of freedom of the full model


Resources:
- [F-test for regression problems](https://en.wikipedia.org/wiki/F-test#Regression_problems)
- [The General Linear F-Test](https://online.stat.psu.edu/stat501/lesson/6/6.2)




In [18]:
dof_num = ((n - 1) - (n - p))
num = (tss - rss) / dof_num
dof_denom = n - p
denom = rss / dof_denom
f_val = num / denom

print(f_val)
# comparing it to statsmodels OLS result
assert f_val == result.fvalue

46.27243958524321


In [19]:
from scipy.stats import f

# f.sf = 1 - f.cdf
f_pval = f.sf(f_val, dof_num, dof_denom)

print(f_pval)
assert f_pval == result.f_pvalue


3.8286490381849547e-62


### Normality Tests

The omnibus is the test statistic of D'Agostino $K^2$ test, which uses the sample kurtosis and skewness to measures the goodness-of-fit departure from normality.

Jarque-Bera test is also a goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution.

Note that the null hypothesis for both test is H0: the data is from a normal distribution.

Resources:
- [D'Agostino $K^2$ test](https://en.wikipedia.org/wiki/D%27Agostino%27s_K-squared_test)
- [Jarque-Bera test](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test)

In [20]:
from scipy.stats import normaltest

omnibus_score, omnibus_pval = normaltest(e)
omnibus_score, omnibus_pval 

(1.5060119895778554, 0.4709487533613883)

In [21]:
from scipy.stats import jarque_bera

jq_score, jq_pval = jarque_bera(e)
jq_score, jq_pval

(1.4037553754122283, 0.4956537465319505)

Skew indicates the symmetry of the distribution. 0 implies perfect symmetry.

Pearson's Kurtosis shows the peakiness of the distribution. A normal distribution's pearson's kurtosis should be close to 3 (or 0 for fisher's kurtosis).


Resources:
- [skew](https://en.wikipedia.org/wiki/Skewness)
- [kurtosis](https://en.wikipedia.org/wiki/Kurtosis)
- [scipy.stats.kurtosis](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html)

In [22]:
from scipy.stats import skew, kurtosis
import plotly.express as px

e_skew, e_kurtosis_pearson = skew(e), kurtosis(e, fisher=False)
px.histogram(e, title=f"hist of e, skew={e_skew: .2f}, pearson kurtosis={e_kurtosis_pearson:.2f}")

In [23]:
result.summary2().tables[-1]

,0,1,2,3
0,Omnibus:,1.506,Durbin-Watson:,2.029
1,Prob(Omnibus):,0.471,Jarque-Bera (JB):,1.404
2,Skew:,0.017,Prob(JB):,0.496
3,Kurtosis:,2.726,Condition No.:,227


### Multicollinearity

#### Condition Number

Condition number is calculated as the the ratio of max_singular_value / min_singlar_value.

The singular values show the rank of the matrix. If any singular value is close to 0, the matrix is not of full rank.

Resources:
- [statsmodels.regression.linear_model.OLSResults.condition_number](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.condition_number.html)
- [Code block computing the condition number](https://github.com/statsmodels/statsmodels/blob/3dfb6c5ca88fdd585a778709ccb59ad9ed33b419/statsmodels/regression/linear_model.py#L1942-L1964). Note that `self._wexog_singular_values` are the singlar values of the whitened X. And OLS whitenning doesn't transform X.

In [149]:
u, s, v = np.linalg.svd(X)
ss = np.sort(s)[::-1]
condition_no = ss[0] / ss[-1]

print(f"sorted singular values: {ss}")
print(f"condition_no: {condition_no}")
assert_almost_equal(condition_no, result.condition_number, decimal=10)

sorted singular values: [21.02379604  2.00604356  1.22160537  1.09816495  0.97748473  0.81374529
  0.77634855  0.73250642  0.65854539  0.27985715  0.09252421]
condition_no: 227.22480485484772


#### VIF

To further identify which columns are linearly dependent, we can compute the variance inflation factor.

Assume the full model uses all p features of X, $\text{VIF}_i = \frac{1}{1-R^2_i}$, where $R_i$ is the coefficient of determination of the model explaining feature `i` using all other features. A high VIF (>5) means that the column is not linearly independent.

In [164]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

for i in range(X.shape[1]):
    print(variance_inflation_factor(X, i))

0.9999999999999998
1.2173065138070078
1.2780710154103618
1.5094373738445481
1.459427777683831
59.202510134318615
39.1933699727707
15.40215600751348
8.890986360329634
10.075967132038336
1.4846226073834823


# Resources

1. [Equations in jupyter notebooks](https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Typesetting%20Equations.html#Motivating-Examples)
1. [Interpreting linear regression summary from statsmodels](https://www.adrian.idv.hk/2021-07-16-statsmodels/)